# Conexión con MongoDB

Cómo efectuar a conexión dende Python.

Podes empregar o escenario de MongoDB para vagrant deste repositorio para montar o servidor e cargar un dataset de exemplo.

## Instalación das librarías

Precisamos instalar con conda `pymongo` e `dnspython`

  `conda install pymongo dnspython`

## Conectando e listando as bases de datos

In [ ]:
from pymongo import MongoClient

#HOST="(...).mongodb.net"
HOST="localhost"
PORT=27017
USERNAME=""
PASSWORD=""

if HOST == 'localhost':
    if not USERNAME:
        cli_mongo = MongoClient(HOST, PORT)
    else:
        cli_mongo = MongoClient(HOST, PORT, USERNAME, PASSWORD)
else:
    cli_mongo = MongoClient(f"mongodb+srv://{USERNAME}:{PASSWORD}@{HOST}/test")

## Listar as bases de datos

In [ ]:
print (cli_mongo.list_database_names())

## Crear unha base de datos

In [ ]:
#Crear base de datos
cli_mongo['BBBDD_a_Usar_o_Crear']
#Hai que crear unha colección e engadir alomenos un documento para ver a BBDD

## Empregar unha base de datos existente

Conectarse a unha base de datos e ver os nomes de coleccións

In [ ]:
db_covid=cli_mongo['covid']
print (db_covid.list_collection_names())

### Inserción dun documento

In [ ]:
coleccion_covid=db_covid['coviditalia']

print ("Antes de insertar, hai: ", coleccion_covid.count_documents({}), "documentos na BBDD")

coleccion_covid.insert_one({
    'codice_regione': '99999',
    'totale_casi': 999999999,
    'stato': 'ITA'
})

print ("Despois de insertar, hai: ", coleccion_covid.count_documents({}), "documentos na BBDD")

### Inserción dunha colección de elementos

In [ ]:
print ("Antes de insertar unha colección de 2 novos documentos, hai: ", coleccion_covid.count_documents({}), "documentos na BBDD")

coleccion_covid.insert_many([ {
    'codice_regione': '999991',
    'totale_casi': 9999999991,
    'stato': 'ITA'
},{
    'codice_regione': '999992',
    'totale_casi': 9999999992,
    'stato': 'ITA'
}])

print ("Depois de insertar unha colección de 2 novos documentos, hai: ", coleccion_covid.count_documents({}), "documentos na BBDD")

### Buscando documentos

In [ ]:
#Buscar por valor concreto
print ("Buscando documentos con exactamente 999999999 casos totales")
aux_documentos = coleccion_covid.find({
    'totale_casi': 999999999
})
for documento in aux_documentos:
    print (documento)

#Buscar usando condiciones (mayor que ese número de casos)
print ("Buscando documentos con más de 999999999 casos totales")
aux_documentos = coleccion_covid.find({
    'totale_casi': {"$gt": 999999999}
})
for documento in aux_documentos:
    print (documento)

#Buscar exactamente un documento (cuidado si hay varios al devolver el primero)
print ("Buscando primer documento con exactamente 9999999991 casos totales")
uno_doc= coleccion_covid.find_one({'totale_casi': 9999999991})
print (uno_doc)


### Borrando documento/s

In [ ]:
#Borrando un documento
print ("Antes de borrar 1 documento, hai: ", coleccion_covid.count_documents({}), "documentos na BBDD")
borrado=coleccion_covid.delete_one({'codice_regione': '999991', 'totale_casi': 9999999991, 'stato': 'ITA'})
print ("Despois de borrar 1 documento, hai: ", coleccion_covid.count_documents({}), "documentos na BBDD (", borrado.deleted_count,") borrado")

#Borrando varios documentos
print ("Antes de borrar varios documentos, hai: ", coleccion_covid.count_documents({}), "documentos na BBDD")
borrados=coleccion_covid.delete_many({'codice_regione': {'$regex': '^99999'}})
print ("Despois de borrar varios documentos, hai: ", coleccion_covid.count_documents({}), 
    "documentos na BBDD (", borrados.deleted_count,") borrados")

### Modificando/Actualizando documentos

In [ ]:
#Primeiro imos meter uns datos de exemplo (os anteriores deberían estar borrados)
coleccion_covid.insert_many([ {
    'codice_regione': '01234567890',
    'totale_casi': 1024,
    'stato': 'ITA'
},{
    'codice_regione': '11234567891',
    'totale_casi': 1024,
    'stato': 'ITA'
}])

#Amosamos os documentos
def amosar_documentos(texto):
    print (texto)
    aux_documentos = coleccion_covid.find({
        'codice_regione': {"$regex": '^[0-1]123456789[0-1]$'}
    })
    for i in aux_documentos:
        print(i)

amosar_documentos("Documentos de exemplo insertados antes do cambio")

#Actualizar un documento con 2048 casos
filtro_busqueda={'codice_regione': '01234567890'}
valores_actualizar = {'totale_casi': 2048}
coleccion_covid.update_one(
    filtro_busqueda,
    {"$set": valores_actualizar
})

amosar_documentos("Documento de exemplo actualizado (un con 2048 casos)")

#Actualizar os dous documentos con 4096 casos
filtro_busqueda={'codice_regione': {"$regex": '^[0-1]123456789[0-1]$'}}
valores_actualizar = {'totale_casi': 4096}
coleccion_covid.update_many(
    filtro_busqueda,
    {"$set": valores_actualizar
})

amosar_documentos("Documentos de exemplo actualizados (ambos con 4096 casos)")

docs_borrados=coleccion_covid.delete_many(filtro_busqueda)

print ("Borrados documentos de proba: ", docs_borrados.deleted_count)